In [1]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import MiniBatchKMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from skimage.feature import local_binary_pattern


dataset_path='Dataset'
RANDOMSEED=1277
num_type=15
label_list = [
    "Agriculture", "Airport", "Beach", "City", "Desert", "Forest",
    "Grassland", "Highway", "Lake", "Mountain", "Parking", "Port",
    "Railway", "Residential", "River"
]

In [2]:
code = pd.read_pickle("D:/temp/index_df_code.pkl")
lbp_features = pd.read_pickle("D:/temp/index_df_lbp_features.pkl")

index_df = pd.concat([code, lbp_features], axis=1)


In [3]:
totsize=len(index_df)
print(totsize)
train_size=int(totsize*0.8)
test_size=int(totsize*0.2)

train_per_code=train_size//num_type
test_per_code=test_size//num_type

train_df=pd.DataFrame()
test_df=pd.DataFrame()
# use train_test_split to split test and train set
for code in index_df['code'].unique():
    # every word, each word is code
    sample_code= index_df[index_df['code'] == code]
    train_samples, test_samples = train_test_split(
        sample_code, 
        test_size=test_per_code, 
        train_size=train_per_code,
        random_state=RANDOMSEED, 
        shuffle=True
    )
 
    train_df = pd.concat([train_df, train_samples], ignore_index=True)
    test_df = pd.concat([test_df, test_samples], ignore_index=True)

print('finish')

12000
finish


In [4]:
def printresult_pro(y_true, y_pred, y_proba, label_names=None, top_n=3):
    print("Evaluation Metrics:")
    print("Accuracy Score:", metrics.accuracy_score(y_true, y_pred))
    print("Recall Score (macro):", metrics.recall_score(y_true, y_pred, average='macro'))
    print("F1 Score (macro):", metrics.f1_score(y_true, y_pred, average='macro'))
    print("Confusion Matrix:")
    print(metrics.confusion_matrix(y_true, y_pred))

    # 2. precision / recall / f1
    print("\nPer-Class Performance:")
    report = metrics.classification_report(y_true, y_pred, target_names=label_names, digits=3)
    print(report)

In [5]:
train_lbp = np.array(train_df['lbp_features'].tolist())
test_lbp = np.array(test_df['lbp_features'].tolist())


pca = PCA(n_components=60)  #
train_lbp = pca.fit_transform(train_lbp)
test_lbp = pca.transform(test_lbp)

scaler = StandardScaler() # scaler
train_lbp = scaler.fit_transform(train_lbp)
test_lbp = scaler.transform(test_lbp)

y_train = np.array(train_df['code'])
y_test = np.array(test_df['code'])


KNN-lbp

In [6]:
knn_lbp = KNeighborsClassifier(n_neighbors=3, weights='distance')

knn_lbp.fit(train_lbp, y_train)
knn_lbp_predict_y = knn_lbp.predict(test_lbp)
knn_clf_predict_y_proba = knn_lbp.predict_proba(test_lbp)



C:\Users\lele1\anaconda3\envs\comp9417\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 系统找不到指定的文件。
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\subprocess.py", line 550, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\subprocess.py", line 1028, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\lele1\anaconda3\envs\comp9417\Lib\subprocess.py", line 1540, in _execute_child


In [7]:
###### result for KNN-sift_features
print("KNN-lbp:")
#printresult(y_test, knn_lbp_predict_y)
printresult_pro(y_test, knn_lbp_predict_y, knn_clf_predict_y_proba,label_names=label_list)


KNN-lbp:
Evaluation Metrics:
Accuracy Score: 0.51625
Recall Score (macro): 0.5162500000000001
F1 Score (macro): 0.5152049190097467
Confusion Matrix:
[[ 91  12   7   2   8   0   5   5   3   6   6   5   3   2   5]
 [ 12  55   2  17   1   0   0  20   0  10   8   0  21   8   6]
 [  5   2  62   3  22   2   0   1  11   7   7  14   1  10  13]
 [  3  11   2  71   2   0   0  12   1   5  14   1  26   9   3]
 [  8   2  21   6  76   0  16   1   2  13   2   1   0   2  10]
 [  0   1   2   0   2 120  14   1   2  10   0   1   0   4   3]
 [  4   1   6   0   9  14 110   3   1   1   0   1   0   2   8]
 [ 12  17   2   9   1   1   0  69   1   4   7   1  16  18   2]
 [  2   3  12   1   4   5   1   0  87  12   0   8   3   2  20]
 [  2  11   3   4  10  11   2   2   6  88   1   0   1   6  13]
 [  3   8   2  15   4   0   1   4   2   1  89   4   6  16   5]
 [  6   4  10   3   0   0   0   2  12   1   7 107   3   3   2]
 [  1  19   0  33   2   0   0  20   1   3   3   1  69   6   2]
 [  2   7   3   5   5   2   0   

SVC-LBP

In [8]:
svc_lbp = SVC(kernel='rbf', random_state=RANDOMSEED,probability=True)
svc_lbp.fit(train_lbp, y_train)
svc_lbp_predict = svc_lbp.predict(test_lbp)
svc_lbp_prob = svc_lbp.predict_proba(test_lbp)


In [9]:
#result for svc-lbp
print("svc-lbp:")
printresult_pro(y_test, svc_lbp_predict, svc_lbp_prob,label_names=label_list)


svc-lbp:
Evaluation Metrics:
Accuracy Score: 0.6075
Recall Score (macro): 0.6075
F1 Score (macro): 0.6055839400608461
Confusion Matrix:
[[103  13   9   4   7   0   2   1   1   5   6   1   3   1   4]
 [ 12  79   1  10   1   1   0  16   0  10  11   0  11   4   4]
 [  7   3  72   3  16   3   2   1   9   5   1  10   0  10  18]
 [  2   8   3  94   1   0   0   8   0   6  14   0  17   5   2]
 [ 10   1  12   4  95   0   4   1   2  19   1   1   0   4   6]
 [  0   0   0   0   1 140  10   0   2   4   0   1   0   0   2]
 [  6   0   7   0   8  17 115   2   0   2   0   0   0   1   2]
 [ 11  20   2   5   1   0   0  76   1   7   3   0  14  19   1]
 [  1   0  12   0   5   3   6   0  98  11   0   5   0   2  17]
 [  0   1   1   2   7   4   3   2   1 122   0   0   3   4  10]
 [  3  13   2  13   1   0   0   6   1   3  97   3   5  10   3]
 [  5   1   7   3   1   0   0   0  12   1   6 119   0   2   3]
 [  2  11   0  32   0   0   0  12   1   8   4   0  85   3   2]
 [  1   7   2   6   2   0   0   7   0   9   8

In [10]:
rf_lbp = RandomForestClassifier(n_estimators=100, random_state=RANDOMSEED)
rf_lbp.fit(train_lbp, y_train)
rf_lbp_predict = rf_lbp.predict(test_lbp)
rf_lbp_prob = rf_lbp.predict_proba(test_lbp)

In [11]:
#result for rf-lbp
print("rf-lbp:")
printresult_pro(y_test, rf_lbp_predict, rf_lbp_prob,label_names=label_list)


rf-lbp:
Evaluation Metrics:
Accuracy Score: 0.58375
Recall Score (macro): 0.58375
F1 Score (macro): 0.5807112508230017
Confusion Matrix:
[[101  18   8   1   6   0   2   3   4   4   5   2   2   1   3]
 [ 15  65   1  12   3   0   0  22   0   9   4   1  13  10   5]
 [  5   3  65   1  17   1   1   1  16   8   9   8   0   9  16]
 [  2  10   1  87   1   0   0   5   1   5  16   0  21   8   3]
 [  9   1  12   3  95   0   5   1   3  16   0   0   0   7   8]
 [  0   0   1   0   2 129  17   0   2   5   0   1   0   3   0]
 [  4   1   4   0  12  14 119   2   0   1   0   0   0   1   2]
 [ 14  20   1   2   1   0   2  76   1   5   5   1  11  19   2]
 [  2   0  13   0   4   2   7   0 107   5   0   5   0   2  13]
 [  2   3   0   2   7   5   3   2   2 115   0   0   2   6  11]
 [  2  10   3  15   1   0   0   6   2   1  93   5   7  12   3]
 [  5   1   9   1   0   0   0   0  13   1   7 116   3   2   2]
 [  2  19   0  28   2   0   0  16   0   7   3   1  75   6   1]
 [  0   6   1   7   2   2   0   5   0  13   